<a href="https://colab.research.google.com/github/sujitpatel22/GANs_Projects/blob/main/FID_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from tensorflow.keras.preprocessing import image
from scipy.linalg import sqrtm
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [2]:
import tarfile
import urllib.request

# Define the URLs for the dataset
images_url = 'https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz'
annotations_url = 'https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz'

# Define the local paths where the files will be saved
images_tar_path = '/content/images.tar.gz'
annotations_tar_path = '/content/annotations.tar.gz'
images_path = '/content/images'
annotations_path = '/content/annotations'

# Download the images and annotations tar files
!wget {images_url} -O {images_tar_path}
!wget {annotations_url} -O {annotations_tar_path}

# Extract the tar files
with tarfile.open(images_tar_path, 'r:gz') as tar:
    tar.extractall(path=images_path)

with tarfile.open(annotations_tar_path, 'r:gz') as tar:
    tar.extractall(path=annotations_path)

# Verify extraction by listing contents of extracted folders
print("Extracted images directory:")
print(os.listdir(images_path))

print("Extracted annotations directory:")
print(os.listdir(annotations_path))

--2024-08-15 18:44:56--  https://thor.robots.ox.ac.uk/~vgg/data/pets/images.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)|129.67.95.98|:443... connected.
HTTP request sent, awaiting response... 308 Permanent Redirect
Location: https://thor.robots.ox.ac.uk/pets/images.tar.gz [following]
--2024-08-15 18:44:57--  https://thor.robots.ox.ac.uk/pets/images.tar.gz
Reusing existing connection to thor.robots.ox.ac.uk:443.
HTTP request sent, awaiting response... 200 OK
Length: 791918971 (755M) [application/octet-stream]
Saving to: ‘/content/images.tar.gz’

/content/images.tar 100%[===================>] 755.23M  21.2MB/s    in 38s     

2024-08-15 18:45:36 (19.6 MB/s) - ‘/content/images.tar.gz’ saved [791918971/791918971]

--2024-08-15 18:45:36--  https://thor.robots.ox.ac.uk/~vgg/data/pets/annotations.tar.gz
Resolving thor.robots.ox.ac.uk (thor.robots.ox.ac.uk)... 129.67.95.98
Connecting to thor.robots.ox.ac

In [3]:
# Path to images and labels
images_path = '/content/images/images'
train_labels_path = annotations_path + '/annotations/trainval.txt'
test_labels_path = annotations_path + '/annotations/test.txt'

# Function to load the dataset
def load_dataset(images_path, labels_path):
    images = []
    labels = []

    # Load labels
    with open(labels_path, 'r') as f:
        lines = f.readlines()

    # Loop through each line in the labels file
    for line in lines:
        image_name = line.strip().split(' ')[0]
        label = 0 if image_name[0].isupper() else 1  # 0 for cats, 1 for dogs

        # Load image
        img_path = os.path.join(images_path, image_name)
        img = tf.io.read_file(f'{img_path}.jpg')
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, (299, 299))

        images.append(img)
        labels.append(label)

    # Convert to TensorFlow tensors
    images = tf.stack(images)
    labels = tf.convert_to_tensor(labels)

    return images, labels

In [4]:
# Load training and test datasets
train_images, train_labels = load_dataset(images_path, train_labels_path)
test_images, test_labels = load_dataset(images_path, test_labels_path)

In [5]:
print(train_images.shape)
print(train_labels.shape)
print(test_images.shape)
print(test_labels.shape)

(3680, 299, 299, 3)
(3680,)
(3669, 299, 299, 3)
(3669,)


In [ ]:
# Display some images to check if they are loaded correctly
plt.figure(figsize=(10,10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(train_images[i].numpy().astype("uint8"))
    plt.title("Cat" if train_labels[i].numpy() == 0 else "Dog")
    plt.axis("off")
plt.show()

In [7]:
# Load the pretrained InceptionV3 model (excluding the top classification layer)
inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))

87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step


In [11]:
# Function to extract features in batches
def extract_features_in_batches(images, batch_size = 4):
    features_list = []

    # Iterate over images in batches
    for i in range(0, len(images), batch_size):
        batch_images = images[i:i + batch_size]
        batch_features = inception_model(batch_images, training=False)
        features_list.append(batch_features)

    # Concatenate all features
    features = tf.concat(features_list, axis=0)
    return features

# Extract features for training and test datasets in batches
train_features = extract_features_in_batches(train_images)
test_features = extract_features_in_batches(test_images)

# Display feature shapes to verify extraction
print("Train features shape:", train_features.shape)
print("Test features shape:", test_features.shape)

KeyboardInterrupt: 

In [9]:
def compute_mean_and_cov(features):
    # Compute the mean and covariance matrix
    mean = np.mean(features, axis = 0)
    covariance = np.cov(features, rowvar = False)
    return mean, covariance

# Compute mean and covariance for training and test datasets
train_mean, train_cov = compute_mean_and_cov(train_features.numpy())
test_mean, test_cov = compute_mean_and_cov(test_features.numpy())

print("Train mean shape:", train_mean.shape)
print("Train covariance shape:", train_cov.shape)
print("Test mean shape:", test_mean.shape)
print("Test covariance shape:", test_cov.shape)

Train mean shape: (2048,)
Train covariance shape: (2048, 2048)
Test mean shape: (2048,)
Test covariance shape: (2048, 2048)


In [10]:
def calculate_fid_score(mu1, sigma1, mu2, sigma2):
    # Compute the difference in means
    mu_diff = mu1 - mu2
    mu_diff_sq = np.dot(mu_diff, mu_diff)

    # Compute the covariance matrix product
    cov_sqrt, _ = sqrtm(sigma1 @ sigma2, disp=False)
    cov_diff = sigma1 + sigma2 - 2 * cov_sqrt

    # Compute the FID score
    fid_score = mu_diff_sq + np.trace(sigma1 + sigma2 - 2 * cov_sqrt)
    return fid_score

# Calculate FID score between train and test features
fid_score = calculate_fid_score(train_mean, train_cov, test_mean, test_cov)

print("FID Score:", fid_score)

FID Score: (1205.1896751501795+9.444140256380244e-19j)
